In [ ]:
from collections import defaultdict
import glob
import os
import re
import xml.etree.ElementTree as et
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
XML_FOLDER = '/Users/ndiamant/Downloads/ecg-rest-and-bike/2019-05-23/raw'

In [ ]:
def heart_rate_rec(root, trends):
    max_hr = int(root.find('./ExerciseMeasurements/MaxHeartRate').text)
    print(root.find('./ExerciseMeasurements/TargetHeartRate').attrib)
    time_hrs = np.array([(float(trend['PhaseTime']), float(trend['HeartRate'])) for trend in trends.values() if trend['PhaseName'] == 'Rest'])
    target_times = np.array([10, 20, 30, 40, 50]) * 1.
    print(max_hr, root.find('./ExerciseMeasurements/MaxHeartRate').text)
    return max_hr - np.interp(target_times, time_hrs[:, 0], time_hrs[:, 1]).mean()
    

def process_raw_ekg(sample_id):
    xml_path = os.path.join(XML_FOLDER, sample_id)
    ecg = glob.glob(xml_path)
    root = et.parse(ecg[0]).getroot()
    trends = defaultdict(dict)
    trend_entry_fields=['HeartRate', 'Load', 'Grade', 'StageName', 'PhaseName', 'Mets', 'VECount', 'PaceCount']

    for trend_entry in root.findall("./TrendData/TrendEntry"):
        idx = int(trend_entry.get("Idx"))
        for field in trend_entry_fields:
            trends[idx][field]=trend_entry.find(field).text
        current_trend_start_time = 60 * int(trend_entry.find("EntryTime/Minute").text) + int(trend_entry.find("EntryTime/Second").text)
        trends[idx]['PhaseTime'] = 60 * int(trend_entry.find("PhaseTime/Minute").text) + int(trend_entry.find("PhaseTime/Second").text)
        trends[idx]['start'] = current_trend_start_time
        if idx > 0:
            trends[idx-1]['end'] = current_trend_start_time
        #last trend has no end
    #protocol
    phase_durations = {}
    for protocol in root.findall("./Protocol/Phase"):
        phase_name = protocol.find("PhaseName").text
        phase_duration = 60 * int(protocol.find("PhaseDuration/Minute").text) + int(protocol.find("PhaseDuration/Second").text)
        phase_durations[phase_name] = phase_duration
#     print(trends)

    # GET LEAD DATA
    counter = 0
    full_ekgs = defaultdict(list)
    full_leads = {}
    fd_start_time = (60 * int(root.find("./FullDisclosure/StartTime/Minute").text) + 
                       float(root.find("./FullDisclosure/StartTime/Second").text))
    fd_sample_rate = int(root.find("./FullDisclosure/SampleRate").text)
    

    for lead_order in root.findall("./FullDisclosure/LeadOrder"):
        full_leads = {i: lead for i, lead in enumerate(lead_order.text.split(','))}

    for full_d in root.findall("./FullDisclosure/FullDisclosureData"):
        for full_line in re.split('\n|\t', full_d.text):
            for sample in re.split(',', full_line):
                if sample == '':
                    continue
                full_ekgs[full_leads[counter % 3]].append(float(sample))
                counter += 1
#     print(fd_start_time, sum(phase_durations.values()), len(full_ekgs['I']) // fd_sample_rate)
#     print(set([trend['PhaseName'] for trend in trends.values()]))
#     print([(trend['PhaseTime'], trend['Load'], trend['HeartRate']) for trend in trends.values() if trend['PhaseName'] == 'Pretest'])
#     print([(trend['PhaseTime'], trend['Load'], trend['HeartRate']) for trend in trends.values() if trend['PhaseName'] == 'Rest'])
# #     print('percent of max hr', root.find('./ExerciseMeasurements/PercentAchievedMaxPredicted').text)
# #     print(root.find('./ExerciseMeasurements/MaxSTStats').text)
#     print(max([trend['start'] for trend in trends.values()]))
#     print('max / min', max([int(trend['HeartRate']) for trend in trends.values()]) / min([int(trend['HeartRate']) for trend in trends.values() if int(trend['HeartRate']) > 0]))
    print(heart_rate_rec(root, trends))

In [ ]:
SAMPLE_IDS = ('1994810_6025_0_0.xml', '1720892_6025_0_0.xml', '5605291_6025_1_0.xml', '5483286_6025_0_0.xml')
for s in SAMPLE_IDS:
    process_raw_ekg(s)